In [1]:
import tableauserverclient as TSC
import os
import pandas as pd
import smtplib
from urllib3 import disable_warnings

In [2]:
ds_config_path = r'C:\Users\abirm.USEREADYTECH\OneDrive - USEReady Technology Private Limited\Desktop\ab\updated scripts\Citi bank opstability\file_watcher_script\Ds_config.xlsx'
file_watcher_path = r"C:\Users\abirm.USEREADYTECH\OneDrive - USEReady Technology Private Limited\Desktop\file watcher"
wait_time_second = 10




In [3]:
# Getting the login credentials

with open ('login_cred.txt','r') as fp:
    lst = fp.readlines()
cred_lst = list()    
for i in lst:
    i = i.replace('\n','')
    cred_lst.append(i)
    
server = cred_lst[0:cred_lst.index('')]
cloud = cred_lst[cred_lst.index('')+1:len(cred_lst)]

server_cred = dict()
for i in server:
    if i.find(':')>=0:
        server_cred[i[0:i.index(':')]] = i[int(i.index(':')+1):len(i)].strip()
        
cloud_cred = dict()
for i in cloud:
    if i.find(':')>=0:
        cloud_cred[i[0:i.index(':')]] = i[int(i.index(':')+1):len(i)].strip()

In [4]:
server_cred

{'url': 'https://urcptableau.useready.com/',
 'username': 'abirm',
 'password': 'Sanju@2001',
 'site': 'DataManagement'}

In [8]:
def login_tableau_tsc(server_cred):
    """
    Checks if their is any data available on the server
    """
    server_url=  server_cred['url']
    username = server_cred['username']            
    password = server_cred['password']                               
    site =  server_cred['site']
    tableau_auth = TSC.TableauAuth(username,password,site)
    server = TSC.Server(server_url,use_server_version=True)
    server.auth.sign_in(tableau_auth)
    return tableau_auth,server

def get_ds_config_file_list(file_path):
    ds_config = pd.read_excel(file_path)
    ds_config['file_name'] = ds_config['datasource_name']+'#'+ds_config['datasource_id']
    file_list = list()
    for i in ds_config.file_name.unique():
        file_list.append(i.replace(' ',''))
    return file_list

def get_file_list(file_path):
    # list to store files
    filelist = []

    # Iterate directory
    for path in os.listdir(file_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(file_path, path)):
            filelist.append(path.split('.txt')[0])
    return filelist

def check_datasource_available(datasource_id,tableau_auth):
    with server.auth.sign_in(tableau_auth):
        all_datasources = []
        cnt = 1000
        page = 1
        while cnt == 1000:
            a_w,p_i = server.datasources.get(req_options=TSC.RequestOptions(pagenumber=page,pagesize=1000))
            cnt = len(a_w)
            page += 1
            all_datasources += a_w
    temp = []
    for ds in all_datasources:
        temp.append(ds.id)
    if datasource_id in temp:
        return True
    else:
        return False
    
def send_notification(message,sender_list):
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.login('abir.meshram.1999@gmail.com', 'dajm uquf gsyg orlq')
    for sender in sender_list:
        server.sendmail('abir.meshram.1999@gmail.com',sender,message)
    server.quit()


In [9]:
tableau_auth,server = login_tableau_tsc(server_cred)
ds_config_file_list = get_ds_config_file_list(ds_config_path)
file_watcher_file_list = get_file_list(file_watcher_path)

In [10]:
ds_config_file_list

['Sample_ds_1#e36ea6b4-9cfa-464b-8395-6ec06ce6d782',
 'Sample_ds_2#36e64912-ddb6-402a-aa94-70cd33e46ec9']

In [11]:
file_watcher_file_list

['Sample_ds_1#e36ea6b4-9cfa-464b-8395-6ec06ce6d782']

In [12]:
with server.auth.sign_in(tableau_auth):
    all_datasources = []
    cnt = 1000
    page = 1
    while cnt == 1000:
        a_w,p_i = server.datasources.get(req_options=TSC.RequestOptions(pagenumber=page,pagesize=1000))
        cnt = len(a_w)
        page += 1
        all_datasources += a_w
        
    all_tasks = []
    cnt = 1000
    page = 1
    while cnt == 1000:
        a_w,p_i = server.tasks.get(req_options=TSC.RequestOptions(pagenumber=page,pagesize=1000))
        cnt = len(a_w)
        page += 1
        all_tasks += a_w
            

In [23]:
for task in all_tasks:
    print(task.id,task.target.type,task.target.id)

19a5db93-c7ed-46a4-8a2f-7e5cd25887f3 datasource ca0c0575-9701-4192-9526-30dd8d25ccd7
01205113-e1ac-4934-92ce-392ab7bb1499 datasource e36ea6b4-9cfa-464b-8395-6ec06ce6d782


In [27]:
with server.auth.sign_in(tableau_auth):
    run = server.tasks.run(all_tasks[0])
    
run

b'<?xml version=\'1.0\' encoding=\'UTF-8\'?><tsResponse xmlns="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api https://help.tableau.com/samples/en-us/rest_api/ts-api_3_15.xsd"><job id="ff384cc2-a525-4f8b-8ac7-95c459feaa0e" mode="Asynchronous" type="RefreshExtract" createdAt="2023-12-08T15:37:17Z"><extractRefreshJob><datasource id="ca0c0575-9701-4192-9526-30dd8d25ccd7" name="Sample - Superstore Published DS"/></extractRefreshJob></job></tsResponse>'

In [21]:
with server.auth.sign_in(tableau_auth):
    for ds in all_datasources:
        ds = server.datasources.get_by_id(ds.id)
        print(ds.has_extracts,ds.name)

False Superstore-Snowflake-Demo
False SuperStore_SF
True Sample - Superstore Published DS
True PostgreSQL Extract Test DS
False Operational Stability Datamodel
False Operational Stability Datamodel V1.1
True Sample_ds_1
True Sample_ds_2


In [14]:
with server.auth.sign_in(tableau_auth):
    for file in ds_config_file_list:
        if file in file_watcher_file_list:
            ds_id = file.split('#')[1]
            status = check_datasource_available(ds_id,tableau_auth)
            if status:
                ds = server.datasources.get_by_id(ds_id)
                if ds.has_extracts:
                    for task in all_tasks:
                        if task.target.type == 'datasource' and task.target.id == ds_id:
                            server.tasks.run(task.id)
                        
                
                    

e36ea6b4-9cfa-464b-8395-6ec06ce6d782
36e64912-ddb6-402a-aa94-70cd33e46ec9


In [19]:
project_name ='file_watcher_location'

In [20]:
for ds in all_datasources:
    if ds.project_name == project_name:
        print(ds.name,ds.id,ds.project_name)

Sample_ds_1 e36ea6b4-9cfa-464b-8395-6ec06ce6d782 file_watcher_location
Sample_ds_2 36e64912-ddb6-402a-aa94-70cd33e46ec9 file_watcher_location


In [16]:
for ts in all_tasks:
    print(ts.id,ts.target.type,ts.target.id)
    server.tasks.run()

19a5db93-c7ed-46a4-8a2f-7e5cd25887f3 datasource ca0c0575-9701-4192-9526-30dd8d25ccd7
01205113-e1ac-4934-92ce-392ab7bb1499 datasource e36ea6b4-9cfa-464b-8395-6ec06ce6d782


1.all the datasource will be extract??
2.we can run extract task
